In [2]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
import statsmodels.stats.api as sms


class DataLoader:
    """
    Class responsible for loading data from an SQLite database.

    Attributes:
        db_path (str): Path to the SQLite database.
        dataframes (dict): Dictionary containing the loaded DataFrames.
    """

    def __init__(self, db_path: str):
        """
        Initialize the DataLoader class with a database path.

        Args:
            db_path (str): Path to the SQLite database.
        """
        self.db_path = db_path
        self.dataframes = {}

    def load_data(self) -> dict:
        """
        Load data from the SQLite database and store it in a dictionary.

        Returns:
            dict: A dictionary whose keys are table names and values are the corresponding DataFrames.
        """
        connection = sqlite3.connect(self.db_path)
        tables = [
            "drivers",
            "fcyphases",
            "laps",
            "qualifyings",
            "races",
            "retirements",
            "starterfields",
        ]
        self.dataframes = {
            table: pd.read_sql_query(f"SELECT * FROM {table}", connection)
            for table in tables
        }
        connection.close()
        return self.dataframes

In [ ]:
class DNF():
    
    # Pour chaque driver/team de l'année en cours 
    # Tu prends les observations sur les races de l'année en cours (failures) et l'année en cours et les années train df (accident) -- On obtient un df filtré sur le driver et les races_id correspondants aux années 
    # -- Observations < X : On n'estime pas sur le driver mais sur l'ensemble des drivers ayant assez d'obs -- Valeurs moyennes
    # -- Observations 
    '''
        1st : Need to get alpha and beta of prior distribution based on the train data of all drivers. To have alpha and beta, we need mu and sigma from the ditribution of
        the dnf probabilities of failures/accident of each drivers
    
        2nd : Need to have for each driver separately : z = is the number of successes / N = the number of trials 
    '''
    def __init__(self, season,len_train_df, dataframes, driver, team):
        self.season = season
        self.len_train_df = len_train_df
        self.dfs = dataframes
        self.driver = driver
        self.team = team
        
    def calculate_accident_probability(self):
        #Probability is estimated by the mean of a beta distribution
        retirements_df=self.dfs["retirements"]
        races_df = self.dfs['races']
        seasons_to_train = [self.season - x for x in range (1, self.len_train_df +1)]
        accidents_per_train_season_all_driver = retirements_df[retirements_df["season"].isin(seasons_to_train)][["season","accidents"]].reset_index(drop=True)
        accidents_per_train_season_driver = retirements_df[(retirements_df["driver_id"]==self.driver)&(retirements_df["season"].isin(seasons_to_train))][["season","accidents"]].reset_index(drop=True)
        number_of_accidents = accidents_per_train_season["accidents"].sum() # Paramètre z
        number_of_races_train = len(races_df[races_df['season'].isin(seasons_to_train)]) # Paramètre N
        return(accidents_per_train_season[["accidents"]])

    def calculate_failure_probability(self):
        pass
        ''''retirements_df=self.dfs["retirements"]
        # merge 
        season_failures=retirements_df[(retirements_df["driver_id"]==season_drivers)&(retirements_df["season"]==self.season)]
        season_failures=season_failures[["season","failures"]]
        return(season_failures)
        # Depend de la team et de la saison
        # On prend les courses de la team sur la saison actuelle et recupere la frequences de failures''''

In [81]:
db_path = "F1_timingdata_2014_2019.sqlite"
data_loader = DataLoader(db_path=db_path)
dataframes = data_loader.load_data()
retirements_df = dataframes['retirements']
retirements_df = retirements_df.fillna(0)
seasons_to_train = [2016 - x for x in range (0, 2 +1)]
merged_df = staterfield_df[['race_id', 'driver_id']].merge(races_df[['id', 'season']], left_on='race_id', right_on='id')
merged_df = merged_df.drop('id', axis=1)
merged_df = merged_df[merged_df['season'].isin(seasons_to_train)]
merged_df = merged_df.groupby('driver_id').agg(count_of_race=('race_id','count'))

df_filtered = retirements_df[retirements_df['season'].isin(seasons_to_train)].copy()
df_filtered = df_filtered.groupby("driver_id").agg(total_accident=('accidents','sum'))
df_filtered = df_filtered.merge(merged_df, on='driver_id')
df_filtered['accident_proportion'] = df_filtered['total_accident'] / df_filtered['count_of_race']
#default_proportion = sum(df_filtered[df_filtered['count_of_race']>20]['total_accident']) / sum(df_filtered[df_filtered['count_of_race']>20]['count_of_race'])
#df_filtered['accident_proportion'] = np.where(df_filtered['count_of_race']>20, df_filtered['accident_proportion'], default_proportion)
mu    = np.mean(df_filtered[df_filtered['count_of_race']>20]['accident_proportion'])
sigma = np.std(df_filtered[df_filtered['count_of_race']>20]['accident_proportion'])

alpha = ((1-mu)/sigma**2 - 1/mu)*mu**2
beta = alpha*(1/mu - 1)
df_filtered['alpha_posterior'] = df_filtered['total_accident'] + alpha
df_filtered['beta_posterior'] = df_filtered['count_of_race'] - df_filtered['total_accident'] + beta

df_filtered['accident_proba'] = df_filtered['alpha_posterior'] / (df_filtered['alpha_posterior'] + df_filtered['beta_posterior'])
df_filtered
#df_filtered.groupby("driver_id").apply(lambda x : merged_df[merged_df['driver_id']==x['driver_id'].iloc[0]]['count_of_race'].iloc[0])

,total_accident,count_of_race,accident_proportion,alpha_posterior,beta_posterior,accident_proba
driver_id,,,,,,
1,1.0,59,0.016949,2.894488,89.435797,0.031349
2,0.0,59,0.000000,1.894488,90.435797,0.020519
3,1.0,59,0.016949,2.894488,89.435797,0.031349
4,2.0,40,0.050000,3.894488,69.435797,0.053109
5,2.0,57,0.035088,3.894488,86.435797,0.043114
6,0.0,19,0.000000,1.894488,50.435797,0.036203
7,6.0,58,0.103448,7.894488,83.435797,0.086439
8,2.0,58,0.034483,3.894488,87.435797,0.042642
9,3.0,59,0.050847,4.894488,87.435797,0.053011


In [73]:
df_filtered

,total_accident,count_of_race,accident_proportion
driver_id,,,
1,0.0,21,0.000000
2,1.0,21,0.047619
4,1.0,21,0.047619
7,1.0,21,0.047619
8,1.0,21,0.047619
11,1.0,21,0.047619
12,0.0,21,0.000000
15,1.0,21,0.047619
16,1.0,21,0.047619


In [52]:
# Charger la base de données
db_path = "F1_timingdata_2014_2019.sqlite"
data_loader = DataLoader(db_path=db_path)
dataframes = data_loader.load_data()

# Charger les datasets pertinents
retirements_df = dataframes['retirements']
races_df = dataframes['races']
starterfields_df = dataframes['starterfields']

# Nettoyer les valeurs NaN
retirements_df = retirements_df.fillna(0)

# Filtrer sur les saisons souhaitées
seasons_to_train = [2016 - x for x in range(0, 2 +1)]

# Filtrer les courses pour les saisons données
races_filtered = races_df[races_df['season'].isin(seasons_to_train)]

# Associer les courses aux pilotes ayant pris le départ
merged_df = starterfields_df.merge(races_filtered, left_on='race_id', right_on='id')

# Compter le nombre total de courses par pilote
total_races_per_driver = merged_df.groupby("driver_id")["race_id"].count().reset_index()
total_races_per_driver.rename(columns={"race_id": "total_races"}, inplace=True)

# Filtrer les abandons pour les saisons spécifiées
dnf_df = retirements_df[retirements_df['season'].isin(seasons_to_train)]
display(total_accidents_per_driver)
# Compter le nombre total d'abandons par pilote
total_accidents_per_driver = dnf_df.groupby("driver_id")["race_id"].count().reset_index()
total_accidents_per_driver.rename(columns={"race_id": "total_accidents"}, inplace=True)

# Fusionner les deux DataFrames sur driver_id
final_df = total_races_per_driver.merge(total_accidents_per_driver, on="driver_id", how="left")

# Remplacer NaN (aucun accident enregistré) par 0
final_df["total_accidents"] = final_df["total_accidents"].fillna(0)

# Calcul du ratio des accidents par pilote
final_df["accident_ratio"] = final_df["total_accidents"] / final_df["total_races"]

# # Afficher les résultats sous forme de tableau
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Accident Ratios per Driver", dataframe=final_df)
total_accidents_per_driver

NameError: name 'total_accidents_per_driver' is not defined

In [45]:
merged_df[merged_df.index==df_filtered['driver_id'].iloc[0]]

,count_of_race
driver_id,


In [91]:
retirements_df = dataframes['retirements']
retirements_df = retirements_df.fillna(0)
df_filtered = retirements_df[retirements_df['season'].isin(seasons_to_train)].copy()
#df_filtered = df_filtered.groupby("driver_id").agg(total_accident=('accidents','sum'))
df_filtered


,season,driver_id,accidents,failures
0,2014,43,0.0,0.0
1,2015,43,0.0,0.0
2,2016,43,0.0,0.0
6,2014,5,0.0,2.0
7,2015,5,1.0,6.0
...,...,...,...,...
259,2015,12,1.0,0.0
260,2016,12,2.0,1.0
264,2014,31,0.0,0.0
265,2015,31,0.0,0.0


In [84]:
dataframes['starterfields']

,race_id,driver_id,team,teamcolor,enginemanufacturer,gridposition,status,resultposition,completedlaps,speedtrap
0,1,1,Mercedes,#00D2BE,Mercedes,1,DNF,19,2,252.8
1,1,2,RedBull,#1E41FF,Renault,2,DQ,22,57,292.7
2,1,3,Mercedes,#00D2BE,Mercedes,3,F,1,57,299.1
3,1,4,McLaren,#FF8700,Mercedes,4,F,2,57,316.9
4,1,5,Ferrari,#DC0000,Ferrari,5,F,4,57,304.5
...,...,...,...,...,...,...,...,...,...,...
2474,121,35,AlfaRomeo,#9B0000,Ferrari,16,F,16,54,334.9
2475,121,11,AlfaRomeo,#9B0000,Ferrari,17,F,13,54,329.6
2476,121,44,Williams,#192c4e,Mercedes,18,F,17,54,329.3
2477,121,45,Williams,#192c4e,Mercedes,19,F,19,53,317.8


In [85]:
dataframes['races']

,id,date,season,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength
0,1,2014-03-16,2014,Melbourne,"A2,A3,I,W",None,57,58,5303.0
1,2,2014-03-30,2014,KualaLumpur,"A1,A2,I,W",None,56,56,5543.0
2,3,2014-04-06,2014,Sakhir,"A2,A3,I,W","Crash GUT in lap 41, SC somewhen in the end of...",57,57,5412.0
3,4,2014-04-20,2014,Shanghai,"A2,A3,I,W",None,54,56,5451.0
4,5,2014-05-11,2014,Catalunya,"A1,A2,I,W",None,66,66,4655.0
...,...,...,...,...,...,...,...,...,...
116,117,2019-10-13,2019,Suzuka,"A2,A3,A4,I,W",None,52,53,5807.0
117,118,2019-10-27,2019,MexicoCity,"A3,A4,A6,I,W",None,71,71,4304.0
118,119,2019-11-03,2019,Austin,"A3,A4,A6,I,W",None,56,56,5513.0
119,120,2019-11-17,2019,SaoPaulo,"A2,A3,A4,I,W",None,71,71,4309.0


In [92]:
df_final = (
    dataframes['starterfields'][['race_id', 'driver_id', 'team']].merge(dataframes['races'][['id', 'season']], left_on='race_id', right_on ='id', how='inner')
     .merge(df_filtered, on=['season', 'driver_id'], how='inner')
)
df_final

,race_id,driver_id,team,id,season,accidents,failures
0,1,1,Mercedes,1,2014,0.0,3.0
1,2,1,Mercedes,2,2014,0.0,3.0
2,3,1,Mercedes,3,2014,0.0,3.0
3,4,1,Mercedes,4,2014,0.0,3.0
4,5,1,Mercedes,5,2014,0.0,3.0
...,...,...,...,...,...,...,...
1234,55,34,ManorMarussia,55,2016,0.0,0.0
1235,56,34,ManorMarussia,56,2016,0.0,0.0
1236,57,34,ManorMarussia,57,2016,0.0,0.0
1237,58,34,ManorMarussia,58,2016,0.0,0.0


In [95]:
df_final[df_final['race_id']==1]

,race_id,driver_id,team,id,season,accidents,failures
0,1,1,Mercedes,1,2014,0.0,3.0
19,1,2,RedBull,1,2014,0.0,2.0
38,1,3,Mercedes,1,2014,0.0,2.0
57,1,4,McLaren,1,2014,0.0,1.0
76,1,5,Ferrari,1,2014,0.0,2.0
95,1,6,ToroRosso,1,2014,0.0,5.0
114,1,7,ForceIndia,1,2014,1.0,1.0
133,1,8,ToroRosso,1,2014,0.0,5.0
152,1,9,Williams,1,2014,3.0,0.0
171,1,10,McLaren,1,2014,0.0,0.0


In [47]:
merged_df = staterfield_df[['race_id', 'driver_id']].merge(races_df[['id', 'season']], left_on='race_id', right_on='id')
merged_df = merged_df.drop('id', axis=1)
merged_df = merged_df[merged_df['season'].isin(seasons_to_train)]
merged_df = merged_df.groupby('driver_id').agg(count_of_race=('race_id','count'))
merged_df

,count_of_race
driver_id,
1,59
2,59
3,59
4,40
5,57
6,19
7,58
8,58
9,59


In [15]:
races_df

,id,date,season,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength
0,1,2014-03-16,2014,Melbourne,"A2,A3,I,W",None,57,58,5303.0
1,2,2014-03-30,2014,KualaLumpur,"A1,A2,I,W",None,56,56,5543.0
2,3,2014-04-06,2014,Sakhir,"A2,A3,I,W","Crash GUT in lap 41, SC somewhen in the end of...",57,57,5412.0
3,4,2014-04-20,2014,Shanghai,"A2,A3,I,W",None,54,56,5451.0
4,5,2014-05-11,2014,Catalunya,"A1,A2,I,W",None,66,66,4655.0
...,...,...,...,...,...,...,...,...,...
116,117,2019-10-13,2019,Suzuka,"A2,A3,A4,I,W",None,52,53,5807.0
117,118,2019-10-27,2019,MexicoCity,"A3,A4,A6,I,W",None,71,71,4304.0
118,119,2019-11-03,2019,Austin,"A3,A4,A6,I,W",None,56,56,5513.0
119,120,2019-11-17,2019,SaoPaulo,"A2,A3,A4,I,W",None,71,71,4309.0


In [14]:
len(races_df[races_df['season'].isin(seasons_to_train)])

59

In [ ]:
df = retirements_df[(retirements_df["season"] == seasons_to_train) & (retirements_df["race_id"] < race_id)]

In [67]:
db_path = "F1_timingdata_2014_2019.sqlite"
data_loader = DataLoader(db_path=db_path)
dataframes = data_loader.load_data()
retirements_df = dataframes['retirements']
races_df = dataframes['races']
seasons_to_train = [2016 - x for x in range (1, 2 +1)]
x = retirements_df[retirements_df["season"].isin(seasons_to_train)]['accidents'].reset_index(drop=True).fillna(0).sum()
number_of_races_train = len(races_df[races_df['season'].isin(seasons_to_train)])
#retirements_df[retirements_df["season"].isin(seasons_to_train)]

df_races_groupby = races_df.groupby(['season'], as_index=False).count()
df_races_groupby
test = df_races_groupby[df_races_groupby['season']==2014]['id'].iloc[0]
df_filtred = retirements_df[retirements_df["season"].isin(seasons_to_train)][['season', 'driver_id', 'accidents']].dropna()
df_group_by = df_filtred.groupby(['driver_id', 'season'], as_index=False).sum()
for season in seasons_to_train :
    df_group_by[f"{season}_mean_driver"] = df_group_by['accidents'] / df_races_groupby[df_races_groupby['season']==season]['id'].iloc[0]
df_group_by[df_group_by['season']==2014]

,driver_id,season,accidents,2015_mean_driver,2014_mean_driver
0,1,2014,0.0,0.000000,0.000000
2,2,2014,0.0,0.000000,0.000000
4,3,2014,0.0,0.000000,0.000000
6,4,2014,0.0,0.000000,0.000000
8,5,2014,0.0,0.000000,0.000000
10,6,2014,0.0,0.000000,0.000000
11,7,2014,1.0,0.052632,0.052632
13,8,2014,0.0,0.000000,0.000000
15,9,2014,3.0,0.157895,0.157895
17,10,2014,0.0,0.000000,0.000000


In [ ]:
def main():
    db_path = "F1_timingdata_2014_2019.sqlite"
    data_loader = DataLoader(db_path=db_path)
    dataframes = data_loader.load_data()

    dnf = DNF(season = 2016,
              len_train_df = 2, 
              dataframes = dataframes, 
              driver_id = 1, 
              team = "Mercedes"
    )

    dnf.calculate_accident_probability()
    print("Average Min Pit Stop Duration:", pit_stop.avg_min_pit_stop_duration)

    calculated_duration = pit_stop.calculate_pit_stop_duration()
    print("Calculated Pit Stop Duration:", calculated_duration)


if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"[ERROR] An exception occurred: {e}")